In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.models import Model
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.utils import load_img, img_to_array
from tensorflow.keras.applications import VGG16

In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    "./data/train",
    target_size=(128,128),
    batch_size=16,
    shuffle=True,
    seed=42,
    color_mode="rgb",
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    "./data/test",
    target_size=(128,128),
    batch_size=16,
    shuffle=True,
    seed=42,
    color_mode="rgb",
    class_mode='categorical')

Found 735 images belonging to 7 classes.
Found 315 images belonging to 7 classes.


In [3]:

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_loss') < 0.05):
            self.model.stop_training = True
    
        
        
            
callbacks = myCallback()

In [4]:
Input_layer = tf.keras.layers.Input(shape=(128,128,3))
CNN = Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.LeakyReLU(alpha=0.1),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.LeakyReLU(alpha=0.1),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    
])(Input_layer)


layers = Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
])

vgg16_model = VGG16(weights='imagenet', include_top=False)(Input_layer)
vgg16_model.trainable = False
vgg16_model = tf.keras.layers.Flatten()(vgg16_model)
vgg16_model = tf.keras.layers.Dense(512, activation='relu')(vgg16_model)




merged = tf.keras.layers.Concatenate()([CNN, vgg16_model])
merged = tf.keras.layers.Dropout(0.5)(merged)
merged = tf.keras.layers.Dense(7, activation='softmax')(merged)

ensemble_model = Model(inputs=Input_layer, outputs=merged)

ensemble_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])
ensemble_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 vgg16 (Functional)             (None, None, None,   14714688    ['input_1[0][0]']                
                                512)                                                              
                                                                                                  
 flatten_2 (Flatten)            (None, 8192)         0           ['vgg16[0][0]']                  
                                                                                              

In [5]:
epochs = 10

history = ensemble_model.fit(x = train_generator,
                    validation_data=test_generator ,
                    batch_size=16,
                    verbose=1, 
                    epochs=epochs,
                    callbacks=[callbacks]
                   )


Epoch 1/10
46/46 [==============================] - 123s 3s/step - loss: 1.9086 - accuracy: 0.2789 - val_loss: 1.1279 - val_accuracy: 0.6190
Epoch 2/10
46/46 [==============================] - 121s 3s/step - loss: 0.9717 - accuracy: 0.6354 - val_loss: 0.6836 - val_accuracy: 0.7810
Epoch 3/10
46/46 [==============================] - 121s 3s/step - loss: 0.6524 - accuracy: 0.7578 - val_loss: 0.4929 - val_accuracy: 0.8603
Epoch 4/10
46/46 [==============================] - 118s 3s/step - loss: 0.5336 - accuracy: 0.7986 - val_loss: 0.3773 - val_accuracy: 0.8794
Epoch 5/10
46/46 [==============================] - 117s 3s/step - loss: 0.6722 - accuracy: 0.7633 - val_loss: 0.4838 - val_accuracy: 0.8254
Epoch 6/10
46/46 [==============================] - 122s 3s/step - loss: 0.4596 - accuracy: 0.8259 - val_loss: 0.3461 - val_accuracy: 0.8952
Epoch 7/10
46/46 [==============================] - 121s 3s/step - loss: 0.4143 - accuracy: 0.8422 - val_loss: 0.4787 - val_accuracy: 0.8444
Epoch 8/10
46

In [6]:
ensemble_model.evaluate(test_generator)

20/20 [==============================] - 10s 501ms/step - loss: 0.3415 - accuracy: 0.8921


[0.3415294289588928, 0.8920634984970093]

In [22]:

def loss_val_graph(history):
    no_epochs = len(history.history['loss'])
    
    plt.subplot(1, 2, 1)
    loss_train = history.history['loss']
    loss_val = history.history['val_loss']
    ep = range(no_epochs)
    plt.plot(ep, loss_train, label='Training loss')
    plt.plot(ep, loss_val, label='validation loss')
    plt.title('Training and Validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    loss_train = history.history['accuracy']
    loss_val = history.history['val_accuracy']
    ep = range(no_epochs)
    plt.plot(ep, loss_train,label='Training accuracy')
    plt.plot(ep, loss_val, label='validation accuracy')
    plt.title('Training and Validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.figure(figsize=(20,8))
    plt.show()

In [23]:
loss_val_graph(history)

AttributeError: 'AxesSubplot' object has no property 'figsize'

<Figure size 432x288 with 0 Axes>

In [ ]:
def single_predict(file):
    test_image=load_img(file, target_size=(128,128))
    test_image=img_to_array(test_image)
    test_image=np.expand_dims(test_image,axis=0)
    result=model.predict(test_image)
    sorted_classes = ['10', '100', '20', '200', '2000', '50', '500']
    return sorted_classes[result.argmax()]

In [ ]:
single_predict("data/single/2000rs.jpg")

In [ ]:
train_generator.class_indices

In [ ]:
# model.save_weights('optimised_CNN')

In [ ]:
# import cv2
# import numpy as np


# url = input()
# cap = cv2.VideoCapture(url+"/video")

# while(True):
#     font = cv2.FONT_HERSHEY_SIMPLEX
#     fontScale = 1
#     color = (255, 0, 0)
#     thickness = 2
    
#     ret, frame = cap.read()
#     cv2.imwrite("cv_clicked.jpg",frame)
#     res = single_predict("cv_clicked.jpg")
#     print(res)
#     cv2.putText(frame,res,(50, 50),font,fontScale,color,thickness)
#     cv2.imshow('frame',frame)
    
    
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()